<a href="https://colab.research.google.com/github/petr0vsk/Fraud/blob/main/Fraud_clusters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Считаем процентили 90 для всех ЮЛ

Метод k-means

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# 1. Загрузка и подготовка данных

def load_and_process_csv(csv_file):
    """
    Загружает данные из CSV, проверяет их корректность и выполняет необходимые преобразования.

    Аргументы:
        csv_file (str): путь к CSV-файлу.

    Возвращает:
        pd.DataFrame: обработанный датафрейм с рассчитанными процентилями и оборотами.
    """
    df = pd.read_csv(csv_file, delimiter=';', header=None)
    df.columns = ['inn', 'org_name', 'amount', 'type', 'code', 'timestamp']

    # Преобразование суммы в числовой формат
    df['amount'] = df['amount'].apply(lambda x: str(x).replace(',', '.'))
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

    # Группировка по INN и расчет percentil 90 и total_payments
    result = df.groupby(['inn', 'org_name']).agg(
        percentile_90=('amount', lambda x: x.quantile(0.9)),
        total_payments=('amount', 'sum')
    ).reset_index()

    return df, result  # Возвращаем и исходный DataFrame для расчетов ложных срабатываний

# 2. Кластеризация с тюнингом параметров

def cluster_businesses_with_tuning(df, n_clusters=3, init='k-means++', n_init=10, max_iter=300):
    """
    Выполняет кластеризацию с тюнингом параметров.

    Аргументы:
        df (pd.DataFrame): датафрейм с данными для кластеризации.
        n_clusters (int): количество кластеров.
        init (str): метод инициализации центроидов ('k-means++' или 'random').
        n_init (int): количество запусков алгоритма.
        max_iter (int): максимальное количество итераций.

    Возвращает:
        список датафреймов: датафреймы для каждого кластера.
    """
    clustering_data = df[['percentile_90', 'total_payments']]
    kmeans = KMeans(n_clusters=n_clusters, init=init, n_init=n_init, max_iter=max_iter, random_state=0)
    df['n_cluster'] = kmeans.fit_predict(clustering_data)

    clusters = []
    for i in range(n_clusters):
        clusters.append(df[df['n_cluster'] == i].copy())

    return clusters

# 3. Оценка модели с использованием силуэтного коэффициента

def evaluate_clustering_with_silhouette(df_list):
    """
    Оценивает качество кластеризации с помощью силуэтного коэффициента.

    Аргументы:
        df_list (list): список датафреймов для каждого кластера.

    Возвращает:
        float: силуэтный коэффициент.
    """
    combined_df = pd.concat(df_list)
    X = combined_df[['percentile_90', 'total_payments']]
    labels = combined_df['n_cluster']

    score = silhouette_score(X, labels)
    return score

# 4. Расчет порогов срабатывания и сохранение результатов

def calculate_and_save_thresholds_with_names(clustered_dfs):
    """
    Вычисляет пороги для каждого кластера и сохраняет результаты.

    Аргументы:
        clustered_dfs (list): список датафреймов для каждого кластера.

    Возвращает:
        dict: словарь с порогами для каждого кластера.
    """
    thresholds = {}

    for i, cluster_df in enumerate(clustered_dfs):
        threshold_value = cluster_df['percentile_90'].quantile(0.9)
        cluster_name = f"cluster_{i}"  # Автоматическая генерация имени кластера
        thresholds[cluster_name] = threshold_value

        cluster_df['threshold'] = threshold_value

        #print(f"Порог для {cluster_name}: {threshold_value}")

    return thresholds

# 5. Расчет ложных срабатываний

def calculate_false_triggers(df_transactions, df_cluster, threshold):
    """
    Рассчитывает количество ложных срабатываний для каждого ИНН в кластере.

    Аргументы:
        df_transactions (pd.DataFrame): датафрейм с транзакциями.
        df_cluster (pd.DataFrame): датафрейм с данными кластера.
        threshold (float): порог для данного кластера.

    Возвращает:
        pd.DataFrame: датафрейм с добавленным столбцом 'errors'.
    """
    # Подсчитываем количество транзакций по каждому ИНН
    transaction_counts = df_transactions.groupby('inn').agg(total_transactions=('amount', 'count')).reset_index()

    merged_df = pd.merge(df_transactions, df_cluster[['inn', 'threshold']], on='inn', how='left')

    # Рассчитываем ложные срабатывания
    merged_df['errors'] = merged_df['amount'] > threshold

    # Считаем количество ложных срабатываний по каждому ИНН
    false_triggers_df = merged_df.groupby('inn').agg(total_errors=('errors', 'sum')).reset_index()

    # Объединяем обратно с кластером и добавляем количество транзакций
    df_cluster = pd.merge(df_cluster, false_triggers_df[['inn', 'total_errors']], on='inn', how='left')
    df_cluster = pd.merge(df_cluster, transaction_counts, on='inn', how='left')
    eps = 0.05  # значение можно настроить в зависимости от данных
    min_samples = 10
    #print(f"Количество ложных срабатываний и транзакций добавлено для кластера: {df_cluster[['inn', 'total_errors', 'total_transactions']].head()}")

    return df_cluster

# 6. Вывод метрик по кластерам

def print_cluster_metrics(all_clusters_df):
    """
    Выводит метрики по кластерам, включая процент ложных срабатываний по каждому кластеру и общий процент.

    Аргументы:
        all_clusters_df (pd.DataFrame): объединенный датафрейм с данными всех кластеров.
    """
    # Подсчет общего количества транзакций и ложных срабатываний по кластерам
    cluster_metrics = all_clusters_df.groupby('n_cluster').agg(
        total_transactions=('total_transactions', 'sum'),
        total_errors=('total_errors', 'sum')
    ).reset_index()

    total_errors_all_clusters = cluster_metrics['total_errors'].sum()
    total_transactions_all_clusters = cluster_metrics['total_transactions'].sum()
    overall_error_rate = (total_errors_all_clusters / total_transactions_all_clusters) * 100 if total_transactions_all_clusters > 0 else 0

    # Вывод метрик по кластерам
    for _, row in cluster_metrics.iterrows():
        error_rate = (row['total_errors'] / row['total_transactions']) * 100 if row['total_transactions'] > 0 else 0
        print(f"Кластер {int(row['n_cluster'])}:")
        print(f"  Всего транзакций: {row['total_transactions']}")
        print(f"  Ложных срабатываний: {row['total_errors']}")
        print(f"  Процент ложных срабатываний: {error_rate:.2f}%")

    # Вывод общего процента ложных срабатываний по всем кластерам
    print(f"Общий процент ложных срабатываний по всем кластерам: {overall_error_rate:.2f}%")


# 7. Основная часть кода

if __name__ == "__main__":
    # Загрузка данных
    file_path = 'fraud_02.2024.csv'
    df_transactions, df_aggregated = load_and_process_csv(file_path)

    # Кластеризация на агрегированных данных
    num_clusters = 3  # Задаем произвольное количество кластеров
    clustered_dfs = cluster_businesses_with_tuning(df_aggregated, n_clusters=num_clusters, init='k-means++', n_init=20, max_iter=500)

    # Оценка силуэтного коэффициента
    silhouette_score_value = evaluate_clustering_with_silhouette(clustered_dfs)
    print(f"Силуэтный коэффициент: {silhouette_score_value}")

    # Расчет порогов для каждого кластера
    thresholds = calculate_and_save_thresholds_with_names(clustered_dfs)

    # Инициализируем список для хранения данных по каждому кластеру
    cluster_info = []
    total_transactions_all_clusters = 0
    total_errors_all_clusters = 0

    # Рассчитываем ложные срабатывания и добавляем общее количество транзакций для каждого кластера
    all_clusters_df = pd.DataFrame()  # Общая таблица для всех кластеров
    for i, (cluster_name, threshold) in enumerate(thresholds.items()):
        df_cluster = clustered_dfs[i]

        # Рассчитываем ложные срабатывания
        df_cluster_with_errors = calculate_false_triggers(df_transactions, df_cluster, threshold)

        # Добавляем к общему DataFrame
        all_clusters_df = pd.concat([all_clusters_df, df_cluster_with_errors], ignore_index=True)

        # Собираем информацию для таблицы
        total_transactions = df_cluster_with_errors['total_transactions'].sum()
        total_errors = df_cluster_with_errors['total_errors'].sum()
        error_rate = (total_errors / total_transactions) * 100 if total_transactions > 0 else 0
        num_rows = df_cluster_with_errors.shape[0]

        # Обновляем общий счетчик транзакций и ошибок
        total_transactions_all_clusters += total_transactions
        total_errors_all_clusters += total_errors

        cluster_info.append({
            'Кластер': cluster_name,
            'Перцентиль 90%': int(round(df_cluster['percentile_90'].quantile(0.9))),
            'Порог срабатывания': int(round(threshold)),
            'Всего транзакций': total_transactions,
            'Ложных срабатываний': total_errors,
            'Процент ложных срабатываний': f"{error_rate:.2f}%",
            'Кол-во строк': num_rows
        })

    # Преобразуем собранные данные в DataFrame для вывода в виде таблицы
    cluster_info_df = pd.DataFrame(cluster_info)

    # Вывод таблицы
    print("\nИнформация по кластерам:")
    print(cluster_info_df.to_string(index=False))

    # Рассчитываем и выводим общий процент ложных срабатываний по всем кластерам
    overall_error_rate = (total_errors_all_clusters / total_transactions_all_clusters) * 100 if total_transactions_all_clusters > 0 else 0
    print(f"\nОбщий процент ложных срабатываний по всем кластерам: {overall_error_rate:.2f}%")




Силуэтный коэффициент: 0.9274107872732689

Информация по кластерам:
  Кластер  Перцентиль 90%  Порог срабатывания  Всего транзакций  Ложных срабатываний Процент ложных срабатываний  Кол-во строк
cluster_0         1000000             1000000             66749                 2149                       3.22%          2208
cluster_1        13686500            13686500              1401                   32                       2.28%             4
cluster_2        11610100            11610100             18895                  139                       0.74%            71

Общий процент ложных срабатываний по всем кластерам: 2.67%


Используем метод DBSCAN

In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.metrics import davies_bouldin_score

# 1. Загрузка и подготовка данных

def load_and_process_csv(csv_file):
    """
    Загружает данные из CSV, проверяет их корректность и выполняет необходимые преобразования.

    Аргументы:
        csv_file (str): путь к CSV-файлу.

    Возвращает:
        pd.DataFrame: обработанный датафрейм с рассчитанными процентилями и оборотами.
    """
    df = pd.read_csv(csv_file, delimiter=';', header=None)
    df.columns = ['inn', 'org_name', 'amount', 'type', 'code', 'timestamp']

    # Преобразование суммы в числовой формат
    df['amount'] = df['amount'].apply(lambda x: str(x).replace(',', '.'))
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

    # Группировка по INN и расчет percentil 90 и total_payments
    result = df.groupby(['inn', 'org_name']).agg(
        percentile_90=('amount', lambda x: x.quantile(0.9)),
        total_payments=('amount', 'sum')
    ).reset_index()

    return df, result  # Возвращаем и исходный DataFrame для расчетов ложных срабатываний

# 2. Кластеризация с использованием DBSCAN

def cluster_businesses_with_dbscan(df, eps=0.5, min_samples=5):
    """
    Выполняет кластеризацию с использованием DBSCAN.

    Аргументы:
        df (pd.DataFrame): датафрейм с данными для кластеризации.
        eps (float): максимальное расстояние между точками для образования кластера.
        min_samples (int): минимальное количество точек для формирования кластера.

    Возвращает:
        список датафреймов: датафреймы для каждого кластера.
    """
    clustering_data = df[['percentile_90', 'total_payments']].values
    db = DBSCAN(eps=eps, min_samples=min_samples)
    df['n_cluster'] = db.fit_predict(clustering_data)

    clusters = []
    unique_clusters = df['n_cluster'].unique()

    for cluster in unique_clusters:
        clusters.append(df[df['n_cluster'] == cluster].copy())

    return clusters, df['n_cluster']

# 3. Оценка модели с использованием Davies-Bouldin Index

def evaluate_clustering_with_davies_bouldin(df, cluster_labels):
    """
    Оценивает качество кластеризации с помощью индекса Davies-Bouldin.

    Аргументы:
        df (pd.DataFrame): датафрейм с данными для оценки.
        cluster_labels (array): метки кластеров.

    Возвращает:
        float: индекс Davies-Bouldin.
    """
    X = df[['percentile_90', 'total_payments']].values
    if len(set(cluster_labels)) > 1:
        score = davies_bouldin_score(X, cluster_labels)
    else:
        score = float('inf')  # Если кластер один или все -1 (шум), Davies-Bouldin не может быть рассчитан
    return score

# 4. Расчет порогов срабатывания и сохранение результатов

def calculate_and_save_thresholds_with_names(clustered_dfs, cluster_names):
    """
    Вычисляет пороги для каждого кластера и сохраняет результаты.

    Аргументы:
        clustered_dfs (list): список датафреймов для каждого кластера.
        cluster_names (list): имена для каждого кластера (например, ['small', 'medium', 'big']).

    Возвращает:
        dict: словарь с порогами для каждого кластера.
    """
    thresholds = {}

    for i, cluster_df in enumerate(clustered_dfs):
        threshold_value = cluster_df['percentile_90'].quantile(0.9)
        thresholds[cluster_names[i]] = threshold_value

        cluster_df['threshold'] = threshold_value

    return thresholds

# 5. Расчет ложных срабатываний

def calculate_false_triggers(df_transactions, df_cluster, threshold):
    """
    Рассчитывает количество ложных срабатываний для каждого ИНН в кластере и добавляет общий счетчик транзакций.

    Аргументы:
        df_transactions (pd.DataFrame): датафрейм с транзакциями.
        df_cluster (pd.DataFrame): датафрейм с данными кластера.
        threshold (float): порог для данного кластера.

    Возвращает:
        pd.DataFrame: датафрейм с добавленными столбцами 'total_errors' и 'total_trans'.
    """
    # Подсчет количества транзакций для каждого ИНН
    trans_counts = df_transactions.groupby('inn').agg(total_trans=('amount', 'count')).reset_index()

    merged_df = pd.merge(df_transactions, df_cluster[['inn', 'threshold']], on='inn', how='left')

    # Рассчет ошибок на основе транзакций
    merged_df['errors'] = merged_df['amount'] > threshold
    false_triggers_df = merged_df.groupby('inn').agg(total_errors=('errors', 'sum')).reset_index()

    # Объединяем обратно с кластером
    df_cluster = pd.merge(df_cluster, false_triggers_df[['inn', 'total_errors']], on='inn', how='left')
    df_cluster = pd.merge(df_cluster, trans_counts[['inn', 'total_trans']], on='inn', how='left')

    # Заполняем возможные пропуски (NaN) нулями
    df_cluster['total_errors'] = df_cluster['total_errors'].fillna(0)
    df_cluster['total_trans'] = df_cluster['total_trans'].fillna(0)

    return df_cluster


# 6. Основная часть кода

if __name__ == "__main__":
    # Загрузка данных
    file_path = 'fraud_02.2024.csv'
    df_transactions, df_aggregated = load_and_process_csv(file_path)
min_cluster_size = 5  #
    # Кластеризация на агрегированных данных с использованием DBSCAN
    eps = 0.01  # значение можно настроить в зависимости от данных
    min_samples = 5
    clustered_dfs, cluster_labels = cluster_businesses_with_dbscan(df_aggregated, eps=eps, min_samples=min_samples)

    # Оценка индекса Davies-Bouldin
    davies_bouldin_score_value = evaluate_clustering_with_davies_bouldin(df_aggregated, cluster_labels)
    print(f"Индекс Davies-Bouldin: {davies_bouldin_score_value}")

    # Определение имен для каждого кластера (кроме шума)
    cluster_names = [f"cluster_{i}" for i in range(len(clustered_dfs))]

    # Расчет порогов для каждого кластера и сохранение файлов
    thresholds = calculate_and_save_thresholds_with_names(clustered_dfs, cluster_names)

    # Инициализируем список для хранения данных по каждому кластеру
    cluster_info = []
    total_transactions_all_clusters = 0
    total_errors_all_clusters = 0

    # Рассчитываем ложные срабатывания и добавляем общее количество транзакций для каждого кластера
    all_clusters_df = pd.DataFrame()  # Общая таблица для всех кластеров
    for i, cluster_name in enumerate(cluster_names):
        threshold = thresholds[cluster_name]
        df_cluster = clustered_dfs[i]

        # Рассчитываем ложные срабатывания и количество транзакций
        df_cluster_with_errors = calculate_false_triggers(df_transactions, df_cluster, threshold)

        # Собираем информацию для таблицы
        total_transactions = df_cluster_with_errors['total_trans'].sum()
        total_errors = df_cluster_with_errors['total_errors'].sum()
        error_rate = (total_errors / total_transactions) * 100 if total_transactions > 0 else 0
        num_rows = df_cluster_with_errors.shape[0]

        # Обновляем общий счетчик транзакций и ошибок
        total_transactions_all_clusters += total_transactions
        total_errors_all_clusters += total_errors

        cluster_info.append({
            'Кластер': cluster_name,
            'Перцентиль 90%': int(round(df_cluster['percentile_90'].quantile(0.9))),
            'Порог срабатывания': int(round(threshold)),
            'Всего транзакций': int(total_transactions),
            'Ложных срабатываний': int(total_errors),
            'Процент ложных срабатываний': f"{error_rate:.2f}%",
            'Кол-во строк': num_rows
        })

    # Преобразуем собранные данные в DataFrame для вывода в виде таблицы
    cluster_info_df = pd.DataFrame(cluster_info)

    # Вывод таблицы
    print("\nИнформация по кластерам:")
    print(cluster_info_df.to_string(index=False))

    # Рассчитываем и выводим общий процент ложных срабатываний по всем кластерам
    overall_error_rate = (total_errors_all_clusters / total_transactions_all_clusters) * 100 if total_transactions_all_clusters > 0 else 0

    # Вывод общего процента ложных срабатываний по всем кластерам
    print(f"\nОбщий процент ложных срабатываний по всем кластерам: {overall_error_rate:.2f}%")


Индекс Davies-Bouldin: 1.4923101373565508

Информация по кластерам:
  Кластер  Перцентиль 90%  Порог срабатывания  Всего транзакций  Ложных срабатываний Процент ложных срабатываний  Кол-во строк
cluster_0         1297000             1297000             87038                 3983                       4.58%          2276
cluster_1          100000              100000                 7                    0                       0.00%             7

Общий процент ложных срабатываний по всем кластерам: 4.58%


Иерархическая кластеризация (дивизонная)

In [1]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import fcluster, linkage
from sklearn.metrics import davies_bouldin_score
from scipy.spatial.distance import pdist

# 1. Загрузка и подготовка данных

def load_and_process_csv(csv_file):
    """
    Загружает данные из CSV, проверяет их корректность и выполняет необходимые преобразования.

    Аргументы:
        csv_file (str): путь к CSV-файлу.

    Возвращает:
        pd.DataFrame: обработанный датафрейм с рассчитанными процентилями и оборотами.
    """
    df = pd.read_csv(csv_file, delimiter=';', header=None)
    df.columns = ['inn', 'org_name', 'amount', 'type', 'code', 'timestamp']

    # Преобразование суммы в числовой формат
    df['amount'] = df['amount'].apply(lambda x: str(x).replace(',', '.'))
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

    # Группировка по INN и расчет percentil 90 и total_payments
    result = df.groupby(['inn', 'org_name']).agg(
        percentile_90=('amount', lambda x: x.quantile(0.9)),
        total_payments=('amount', 'sum')
    ).reset_index()

    return df, result  # Возвращаем и исходный DataFrame для расчетов ложных срабатываний

# 2. Иерархическая дивизионная кластеризация

def divisive_clustering(df, metric='euclidean', min_cluster_size=2, n_clusters=3):
    """
    Выполняет иерархическую дивизионную кластеризацию.

    Аргументы:
        df (pd.DataFrame): датафрейм с данными для кластеризации.
        metric (str): метрика расстояния (например, 'euclidean', 'manhattan').
        min_cluster_size (int): минимальный размер кластера.
        n_clusters (int): целевое количество кластеров.

    Возвращает:
        list: список датафреймов для каждого кластера.
    """
    # Вычисление матрицы расстояний
    distances = pdist(df[['percentile_90', 'total_payments']], metric=metric)

    # Применение метода иерархической кластеризации
    Z = linkage(distances, method='ward')  # Используем метод Уорда для минимизации внутрикластерной дисперсии

    # Получаем кластеры
    labels = fcluster(Z, t=n_clusters, criterion='maxclust')
    df['n_cluster'] = labels

    clusters = []
    for cluster_label in np.unique(labels):
        cluster_df = df[df['n_cluster'] == cluster_label]
        if len(cluster_df) >= min_cluster_size:
            clusters.append(cluster_df.copy())

    return clusters, labels

# 3. Оценка качества кластеризации с помощью Davies-Bouldin Index

def evaluate_clustering_with_davies_bouldin(df, cluster_labels):
    """
    Оценивает качество кластеризации с помощью индекса Davies-Bouldin.

    Аргументы:
        df (pd.DataFrame): датафрейм с данными для оценки.
        cluster_labels (array): метки кластеров.

    Возвращает:
        float: индекс Davies-Bouldin.
    """
    X = df[['percentile_90', 'total_payments']].values
    if len(set(cluster_labels)) > 1:
        score = davies_bouldin_score(X, cluster_labels)
    else:
        score = float('inf')  # Если кластер один, Davies-Bouldin не может быть рассчитан
    return score

# 4. Расчет порогов срабатывания и ложных срабатываний

def calculate_false_triggers(df_transactions, df_cluster, threshold):
    """
    Рассчитывает количество ложных срабатываний для каждого ИНН в кластере.

    Аргументы:
        df_transactions (pd.DataFrame): датафрейм с транзакциями.
        df_cluster (pd.DataFrame): датафрейм с данными кластера.
        threshold (float): порог для данного кластера.

    Возвращает:
        pd.DataFrame: датафрейм с добавленными столбцами 'total_errors' и 'total_trans'.
    """
    # Подсчет количества транзакций для каждого ИНН
    trans_counts = df_transactions.groupby('inn').agg(total_trans=('amount', 'count')).reset_index()

    merged_df = pd.merge(df_transactions, df_cluster[['inn', 'threshold']], on='inn', how='left')

    # Рассчет ошибок на основе транзакций
    merged_df['errors'] = merged_df['amount'] > threshold
    false_triggers_df = merged_df.groupby('inn').agg(total_errors=('errors', 'sum')).reset_index()

    # Объединяем обратно с кластером
    df_cluster = pd.merge(df_cluster, false_triggers_df[['inn', 'total_errors']], on='inn', how='left')
    df_cluster = pd.merge(df_cluster, trans_counts[['inn', 'total_trans']], on='inn', how='left')

    # Заполняем возможные пропуски (NaN) нулями
    df_cluster['total_errors'] = df_cluster['total_errors'].fillna(0)
    df_cluster['total_trans'] = df_cluster['total_trans'].fillna(0)

    return df_cluster

# 5. Основная часть кода

if __name__ == "__main__":
    # Загрузка данных
    file_path = 'fraud_02.2024.csv'
    df_transactions, df_aggregated = load_and_process_csv(file_path)

    # Применение иерархической дивизионной кластеризации
    metric = 'euclidean'  # Выбор метрики расстояния
    min_cluster_size = 5  # Минимальный размер кластера
    n_clusters = 5  # Целевое количество кластеров
    clustered_dfs, cluster_labels = divisive_clustering(df_aggregated, metric=metric, min_cluster_size=min_cluster_size, n_clusters=n_clusters)

    # Оценка индекса Davies-Bouldin
    davies_bouldin_score_value = evaluate_clustering_with_davies_bouldin(df_aggregated, cluster_labels)
    print(f"Индекс Davies-Bouldin: {davies_bouldin_score_value}")

    # Расчет ложных срабатываний для каждого кластера
    cluster_info = []
    total_transactions_all_clusters = 0
    total_errors_all_clusters = 0

    for i, df_cluster in enumerate(clustered_dfs):
        # Определяем порог срабатывания и добавляем его в DataFrame
        threshold = df_cluster['percentile_90'].quantile(0.9)
        df_cluster['threshold'] = threshold  # Добавляем порог в DataFrame

        # Рассчитываем ложные срабатывания
        df_cluster_with_errors = calculate_false_triggers(df_transactions, df_cluster, threshold)

        total_transactions = df_cluster_with_errors['total_trans'].sum()
        total_errors = df_cluster_with_errors['total_errors'].sum()
        error_rate = (total_errors / total_transactions) * 100 if total_transactions > 0 else 0
        num_rows = df_cluster_with_errors.shape[0]

        total_transactions_all_clusters += total_transactions
        total_errors_all_clusters += total_errors

        cluster_info.append({
            'Кластер': f'cluster_{i}',
            'Перцентиль 90%': int(round(threshold)),
            'Порог срабатывания': int(round(threshold)),
            'Всего транзакций': int(total_transactions),
            'Ложных срабатываний': int(total_errors),
            'Процент ложных срабатываний': f"{error_rate:.2f}%",
            'Кол-во строк': num_rows
        })

        # Сохранение каждого кластера в отдельный Excel-файл
        file_name = f'cluster_{i}_data_with_errors.xlsx'
        df_cluster_with_errors.to_excel(file_name, index=False)
        print(f"Файл для кластера '{i}' сохранен как {file_name}")

    # Преобразуем собранные данные в DataFrame для вывода в виде таблицы
    cluster_info_df = pd.DataFrame(cluster_info)

    # Вывод информации по кластерам
    print("\nИнформация по кластерам:")
    print(cluster_info_df.to_string(index=False))

    # Вывод общего процента ложных срабатываний
    overall_error_rate = (total_errors_all_clusters / total_transactions_all_clusters) * 100 if total_transactions_all_clusters > 0 else 0
    print(f"\nОбщий процент ложных срабатываний по всем кластерам: {overall_error_rate:.2f}%")


Индекс Davies-Bouldin: 0.37424250455051966
Файл для кластера '0' сохранен как cluster_0_data_with_errors.xlsx
Файл для кластера '1' сохранен как cluster_1_data_with_errors.xlsx
Файл для кластера '2' сохранен как cluster_2_data_with_errors.xlsx

Информация по кластерам:
  Кластер  Перцентиль 90%  Порог срабатывания  Всего транзакций  Ложных срабатываний Процент ложных срабатываний  Кол-во строк
cluster_0          737700              737700             53745                 1477                       2.75%          2092
cluster_1         7364975             7364975             19547                  154                       0.79%           146
cluster_2        23940000            23940000             12352                   47                       0.38%            41

Общий процент ложных срабатываний по всем кластерам: 1.96%
